In [1]:
!pip install -q transformers datasets accelerate bitsandbytes wandb peft trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.3/318.3 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [16]:
import os
import json
import torch
import wandb
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainerCallback
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM, SFTConfig
from peft import LoraConfig, get_peft_model, LoraConfig, TaskType


def formatting_prompts_func(data):
    return [f"### Question: {q}\n ### Answer: {a}" for q, a in zip(data['instruction'], data['output'])]

# LoRA configurations
LORA_R_VALUES = [8, 128, 256]

class WandbLoggingCallback(TrainerCallback):

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            if "loss" in logs:
                wandb.log({"train/loss": logs["loss"], "step": state.global_step})


def train_with_lora(lora_r):

    # Initialize wandb
    wandb.init(project='Hanghae99_week8_basic',reinit=True, name=f"rank {lora_r}")

    # Model and dataset
    MODEL_NAME = "facebook/opt-350m"
    DATASET_NAME = "sahil2801/CodeAlpaca-20k"

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.model_max_length = 128

    lora_dropout: float = 0.1
    lora_alpha: int = 32
    # Load dataset
    dataset = load_dataset(DATASET_NAME, split="train")
    dataset = dataset.train_test_split(test_size=0.2, seed=42)
    train_dataset, eval_dataset = dataset['train'], dataset['test']

    # Data collator
    response_template = " ### Answer:"
    collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)


    # Load base model
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto")

    target_modules = set()

    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):
            names = name.split('.')
            target_modules.add(names[0] if len(names) == 1 else names[-1])

    if "lm_head" in target_modules:  # needed for 16-bit
        target_modules.remove("lm_head")

    target_modules = list(target_modules)

    # Apply LoRA
    lora_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=lora_r,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        target_modules=target_modules
    )
    model = get_peft_model(model, lora_config)

    training_config = SFTConfig(
        output_dir=f"./lora_r_{lora_r}_output",
        save_total_limit=1,
        logging_steps=500,
        eval_steps=500,
        max_steps=5000,
        fp16=True,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        save_steps=500,
        save_strategy="steps",
        logging_strategy="steps",
        evaluation_strategy="steps",
    )

    trainer = SFTTrainer(
        model=model,
        args=training_config,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        formatting_func=formatting_prompts_func,
        data_collator=collator,
        callbacks=[WandbLoggingCallback()]
    )

    # Train
    trainer.train()

    # Log metrics
    metrics = trainer.state.log_history
    wandb.log({"train_metrics": metrics})

    # Memory usage
    print('Max Alloc:', round(torch.cuda.max_memory_allocated(0)/1024**3, 1), 'GB')
    wandb.log({"max_memory_GB": round(torch.cuda.max_memory_allocated(0)/1024**3, 1)})

    wandb.finish()

# Train with different LoRA ranks
for r in LORA_R_VALUES:
    train_with_lora(r)
    torch.cuda.empty_cache()  # GPU 메모리 정리



eval/loss,██▁
eval/mean_token_accuracy,▁▁█
eval/runtime,▁▁▁▁▁▁▁███
eval/samples_per_second,███████▁▁▁
eval/steps_per_second,███████▁▁▁
eval_loss,█▁
learning_rate,█▁
loss,█▁
step,▁
train/epoch,▁▁▁▁▁▂▂▄▄▅▅▇▇██▂▂▂▂▄▄
train/global_step,▁▁▁▁▁▂▂▄▄▅▅▇▇██▂▂▂▂▂▂▂▄▄▄▄


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Tokenizing train dataset:   0%|          | 0/16017 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/16017 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/4005 [00:00<?, ? examples/s]

...

Tokenizing eval dataset:   0%|          | 0/4005 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/4005 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
500,1.847500,1.653809
1000,1.710500,1.565601
1500,1.571900,1.517401
2000,1.602400,1.484538
2500,1.549500,1.465551
3000,1.509500,1.447354
3500,1.524200,1.434394
4000,1.510800,1.425224
4500,1.459500,1.420431
5000,1.490400,1.418031


Max Alloc: 13.5 GB


eval/loss,█▅▄▃▂▂▁▁▁▁
eval/mean_token_accuracy,▁▃▅▆▇▇████
eval/runtime,▃▄▃▄▅▁▄▅█▇
eval/samples_per_second,▆▅▆▅▄█▅▄▁▂
eval/steps_per_second,▆▅▆▅▄█▅▄▁▂
max_memory_GB,▁
step,▁▂▃▃▄▅▆▆▇█
train/epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/global_step,▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇██████
train/grad_norm,▃▁▃▂▁▂▃▄█▄
train/learning_rate,█▇▆▆▅▄▃▃▂▁
